In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os 
import functions


# the data is stored on the external ssd 
# the output should go to the local folder 


In [ ]:
''' to count floods (potential debris flows) use dfspot column 
    no need to do it for all the scenarious,
    just the TL case (meaning 1 per land cover), since its supposed to be the same 
    save as the "location_langcover_dfspot"
    then calculate a difference between dfspot and dfs -> that will be the floods 
'''

In [ ]:
'''
    folders to use
    - langcover1: 
    - landcover2
    - landcover3
    - landcover4
'''

# per year



In [ ]:

landcover_idx = 4
location = 'mustang'

column = 'dfspot'
freq = 'year'


# --- this can be anything ---

# method = 'daily'
# percentile = '40percent'
landcover = f'landcover{landcover_idx}'


# path to data
# folder_path = f'/Volumes/Extreme SSD/202409_paper2_modelruns/30years/SL_{method}/{landcover_idx}landcover_{percentile}/{location}_climate_cut'
folder_path = f'/Volumes/Extreme SSD/202409_paper2_modelruns/30years/{landcover_idx}landcover/{location}_climate_cut'

# output_path = f'/Users/varyabazilova/Desktop/paper2/202406_modelruns/30years/dfspot_count/{location}/'

print('folder with data:\n',folder_path) 
print('  ')
print('output folder:\n',output_path) 
print('   ')
print('landcover id:', landcover_idx)

In [ ]:
%%time

elevation = pd.read_csv(f'/Users/varyabazilova/Desktop/paper2/downscaling_simple/coordinates_and_elevation_with_labels_{location}.csv')[['cellnr2','band_data']] 

result_df = pd.DataFrame()


# Iterate over the files in the folders
for folder_name in os.listdir(folder_path):
    # skip all csvs and . files
    if folder_name.endswith('.csv'):
        continue
    if folder_name.startswith('.'):
        continue 
    
    file_path = os.path.join(folder_path, folder_name, 'Sediment.out') #create a path to file 
    
    if os.path.isfile(file_path): #check if its there 
        # read file 
        output_df = pd.read_csv(file_path)
        # count dfs per given time 
        output_df = functions.count_dfs_per_time(output_df, column, 'year')
        
        # take the COUNT column 
        name_column = output_df['count']
        # rename the columns 
        column_name = f'{column}_{folder_name}'
        result_df[column_name] = name_column
        result_df.columns = [col[-3:] for col in result_df.columns]


annaul_df_count = functions.add_elevation_to_df_count(result_df, elevation, freq)


annaul_df_count.to_csv(output_path + f'{location}_annual_{column}_count_{landcover}.csv')


# per month 

In [ ]:

landcover_idx = 2
location = 'mustang'

column = 'dfspot'
freq = 'month'


# --- this can be anything ---

method = 'daily'
percentile = '20percent'
landcover = f'landcover{landcover_idx}'



# path to data on the external ssd 
# folder_path = f'/Volumes/Extreme SSD/202409_paper2_modelruns/30years/SL_{method}/{landcover_idx}landcover_{percentile}/{location}_climate_cut'
# output_folder = f'/Users/varyabazilova/Desktop/paper2/202406_modelruns/30years/SL_{method}/output_{percentile}/'

folder_path = f'/Volumes/Extreme SSD/202409_paper2_modelruns/30years/{landcover_idx}landcover/{location}_climate_cut'
output_folder = f'/Users/varyabazilova/Desktop/paper2/202406_modelruns/30years/dfspot_count/{location}/'



print('folder with data:\n',folder_path) 
print('  ')
print('output folder:\n',output_folder) 
print('  ')
print('landcover id:', landcover_idx)

elevation = pd.read_csv(f'/Users/varyabazilova/Desktop/paper2/downscaling_simple/coordinates_and_elevation_with_labels_{location}.csv')[['cellnr2','band_data']] 


In [ ]:
%%time



result_df = pd.DataFrame()


# Iterate over the files in the folders
for folder_name in os.listdir(folder_path):
    # skip all csvs and . files
    if folder_name.endswith('.csv'):
        continue
    if folder_name.startswith('.'):
        continue 
    
    file_path = os.path.join(folder_path, folder_name, 'Sediment.out') #create a path to file 
    
    if os.path.isfile(file_path): #check if its there 
        # read file 
        output_df = pd.read_csv(file_path)
        # count dfs per given time 
        output_df = functions.count_dfs_per_time(output_df, column, freq)
        
        # take the COUNT column 
        name_column = output_df['count']
        # rename the columns 
        column_name = f'{column}_{folder_name}'
        result_df[column_name] = name_column
        result_df.columns = [col[-3:] for col in result_df.columns]


df_count = functions.add_elevation_to_df_count(result_df, elevation, freq)




df_count.to_csv(output_folder + f'{location}_monthly_{column}_count_{percentile}_{landcover}.csv')



In [ ]:
# df_count

# ------  another attempt for monthly count

In [66]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os 
import functions



def count_dfs_per_time(sediments, column):
    '''
    sediments - sediment output file
    column - column of interest (e.g. dfs)
    '''

    sediments['D'] = pd.to_datetime(sediments.D)
    sediments['year'] = sediments['D'].dt.year
    sediments['month'] = sediments['D'].dt.month
    sediments = sediments.set_index('D')
    sediments['count'] = sediments[column].apply(lambda x: 1 if x != 0 else 0)
    sym = sediments.groupby(['year', 'month']).sum()
    return sym



def add_elevation_to_dfs_count_monthly(df_count, elevation):
    elevation = elevation.T
    elevation_list = elevation.loc['cellnr2'].tolist()
    
    df_count =df_count[elevation_list]
    
    if elevation_list == df_count.columns.tolist():
        print("default land cover. monthly data: same")
    else:
        print("not the same")
    
    df_count.columns = elevation.loc['band_data']
    df_count = df_count.reset_index()
    df_count['date_id'] = df_count['year'].astype(str) + "_" + df_count['month'].astype(str)

    return df_count

In [114]:

landcover_idx = 4
location = 'langtang'

method = 'once' 
percentile = '40percent'


column = 'dfs'
freq = 'month'





landcover = f'landcover{landcover_idx}'

# folder_path = f'/Volumes/Extreme SSD/202409_paper2_modelruns/30years/{landcover_idx}landcover/{location}_climate_cut'
folder_path = f'/Volumes/Extreme SSD/202409_paper2_modelruns/30years/SL_daily/{landcover_idx}landcover_{percentile}/{location}_climate_cut'
output_folder = f'/Users/varyabazilova/Desktop/paper2/202406_modelruns/30years/SL_{method}/output_{percentile}/'



# folder_path = f'/Volumes/Extreme SSD/202409_paper2_modelruns/30years/{landcover_idx}landcover/{location}_climate_cut'
# output_folder = f'/Users/varyabazilova/Desktop/paper2/202406_modelruns/30years/dfspot_count/{location}2/'



print('folder with data:\n', folder_path) 
print('  ')
print('output folder:\n', output_folder) 
print('  ')
print('landcover id:', landcover_idx)

elevation = pd.read_csv(f'/Users/varyabazilova/Desktop/paper2/downscaling_simple/coordinates_and_elevation_with_labels_{location}.csv')[['cellnr2','band_data']] 


folder with data:
 /Volumes/Extreme SSD/202409_paper2_modelruns/30years/SL_daily/4landcover_40percent/langtang_climate_cut
  
output folder:
 /Users/varyabazilova/Desktop/paper2/202406_modelruns/30years/SL_once/output_40percent/
  
landcover id: 4


In [115]:
%%time

print('landcover id:', landcover_idx)


result_df = pd.DataFrame()


# Iterate over the files in the folders
for folder_name in os.listdir(folder_path):
    # skip all csvs and . files
    if folder_name.endswith('.csv'):
        continue
    if folder_name.startswith('.'):
        continue 
    
    file_path = os.path.join(folder_path, folder_name, 'Sediment.out') #create a path to file 
    
    if os.path.isfile(file_path): #check if its there 
        # read file 
        output_df = pd.read_csv(file_path)
        # count dfs per given time 
        # output_df = functions.count_dfs_per_time(output_df, column, freq)
        output_df = count_dfs_per_time(output_df, column)
        
        # take the COUNT column 
        name_column = output_df['count']
        # rename the columns 
        column_name = f'{column}_{folder_name}'
        result_df[column_name] = name_column
        result_df.columns = [col[-3:] for col in result_df.columns]


df_count = add_elevation_to_dfs_count_monthly(result_df, elevation)

# df_count.to_csv(output_folder + f'{location}_monthly_{column}_count_{landcover}.csv')

df_count.to_csv(output_folder + f'{location}_monthly_{column}_count_{percentile}_{landcover}.csv')


landcover id: 4
default land cover. monthly data: same
CPU times: user 20 s, sys: 638 ms, total: 20.7 s
Wall time: 21 s


In [ ]:
def add_elevation_to_dfs_count_monthly(df_count, elevation):
    elevation = elevation.T
    elevation_list = elevation.loc['cellnr2'].tolist()
    
    df_count =df_count[elevation_list]
    
    if elevation_list == df_count.columns.tolist():
        print("default land cover. monthly data: same")
    else:
        print("not the same")
    
    df_count.columns = elevation.loc['band_data']
    df_count = df_count.reset_index()
    df_count['date_id'] = df_count['year'].astype(str) + "_" + df_count['month'].astype(str)

    return df_count

In [ ]:
test = add_elevation_to_dfs_count_monthly(result_df, elevation)

In [ ]:
test

In [ ]:
elv = elevation.T
elevation_list = elv.loc['cellnr2'].tolist()

In [ ]:
result_df =result_df[elevation_list]

In [ ]:
elevation_list == result_df.columns.tolist()

In [ ]:
result_df1

In [ ]:
    
    # reorder columns of the df count 
    df_count = df_count[elevation_list]
    # check if they are the same
    if elevation_list == df_count.columns.tolist():
        print("default land cover. monthly data: same")
    else:
        print("not the same")
    # rename columns 
    df_count.columns = elevation.loc['band_data']

In [ ]:
def add_elevation_to_df_count(df_count, elevation):
    '''
    df_count - df with count of dfs per given time
    elevation - df with elevation values per cell id 
    '''
    elevation = elevation.transpose()
    elevation_list = elevation.loc['cellnr2'].tolist()
    
    # reorder columns of the df count 
    df_count = df_count[elevation_list]
    # check if they are the same
    if elevation_list == df_count.columns.tolist():
        print("default land cover. monthly data: same")
    else:
        print("not the same")
    # rename columns 
    df_count.columns = elevation.loc['band_data']
    # df_count = df_count.reset_index()

    # if freq == 'year':
    #     df_count['D'] = df_count.D.dt.year
    # if freq == 'month':
    #     df_count['D_year'] = df_count.D.dt.year
    #     df_count['D_month'] = df_count.D.dt.month
    
    return df_count

In [ ]:

df_count['D_year'] = df_count.D.dt.year

In [ ]:
are_values_equal = (df_count.values == df_count2.values).all()







In [ ]:
# test with 1 file 
test = pd.read_csv('/Volumes/Extreme SSD/202409_paper2_modelruns/30years/SL_daily/1landcover_20percent/langtang_climate_cut/12a/Sediment.out')

In [ ]:
test

In [ ]:
test1 = count_dfs_per_time(test, 'dfspot')

In [ ]:


name_column = output_df['count']
# rename the columns 
column_name = f'{column}_{folder_name}'
result_df[column_name] = name_column
result_df.columns = [col[-3:] for col in result_df.columns]


df_count = functions.add_elevation_to_df_count(result_df, elevation, freq)
